In [ ]:
%pip install ruprompts

In this tutorial we show how to load and use the pretrained prompts from HuggingFace Hub.

In [ ]:
# although not used directly, this import is necessary,
# since it adds our custom pipelines to transformers
import ruprompts

from transformers import pipeline

Let's first load the joke prompt:

In [ ]:
ppln_joke = pipeline("text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_joke", device=0)

Note that we didn't specify the `model=` and `tokenizer=` arguments. They are inferred from pretrained prompt config.

Using the pipeline with prompt is almost identical to using the regular text generation pipeline:

In [ ]:
ppln_joke("Говорит как-то дуб вороне")

Although it is more convenient to omit model arguments, it leads to reinstantiation of the same model each time we create a pipeline. An easy fix is to create model and tokenizer once and then pass it to all the pipelines:

In [ ]:
from transformers import GPT2LMHeadModel, AutoTokenizer

model_id = "ai-forever/rugpt3large_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

It will speed up loading the proverb pipeline:

In [ ]:
ppln_proverb = pipeline("text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_proverb", model=model, tokenizer=tokenizer, device=0)
ppln_proverb("Сколько ни")

Loading and using text2text prompts is almost identical:

In [ ]:
ppln_detox = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_detox_russe", model=model, tokenizer=tokenizer, device=0)
ppln_detox({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"})

In [ ]:
long_text = """Несмотря на споры, большинство учёных сошлись во мнении, что кошка является полуодомашненным животным, то есть она способна на сосуществование с человеком, но, потеряв с ним контакт, легко возвращаются к дикому образу существования. Хотя у кошки наблюдаются генетические изменения в сравнении с диким предком, эта разница в 10 раз меньше, чем у собак с волками. Учёные считают, что дикая кошка действительно могла сама прийти к человеку, чтобы питаться грызунами, а такие отношения характеризовались как соседские, и уже через несколько тысяч лет люди сами стали одомашнивать маленьких хищников. Это также, вероятно, объясняет, почему модель поведения кошки почти не изменилась; при одомашнивании собаки из волка человек изменил её образ жизни и среду обитания, кошка же претерпела минимальные изменения Кошка сумела сохранить модель поведения, присущую её диким предкам. Она почти так же хорошо охотится, как дикая кошка, но в то же время способна мирно сосуществовать с человеком, проявлять к нему эмоциональную привязанность, нежность или даже выказывать игривое поведение."""
ppln_summary = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_summarization_mlsum", model=model, tokenizer=tokenizer, device=0)

ppln_summary(long_text, num_beams=2, num_return_sequences=5)

In [ ]:
ppln_summary = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_title_mlsum", model=model, tokenizer=tokenizer, device=0)
ppln_summary(long_text, num_beams=2, num_return_sequences=5)

In some cases pipeline takes multiple arguments:

In [ ]:
context = """В 1997 году Шмидхубер и Сепп Хохрайтер опубликовали работу, описывающую рекуррентную нейронную сеть, которую авторы назвали «Долгая краткосрочная память». В 2015 году эта архитектура была использована в новой реализации распознавания речи в программном обеспечении компании Google для смартфонов.

Исследования Шмидхубера также включают в себя генерализации колмогоровской сложности и метрики «скорость важна» (Speed Prior), создание концепции Машины Гёделя.

В 2014 году Шмидхубер основал компанию Nnaisense для работы в сфере коммерческого применения технологий искусственного интеллекта в таких областях как финансы, тяжёлая промышленность и самоуправляемый автотранспорт. Сепп Хохрайтер и Яан Таллинн занимают в компании пост советников."""

ppln_qa = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_qa_sberquad", model=model, tokenizer=tokenizer, device=0)
ppln_qa({"context": context, "question": "С кем Шмидхубер опубликовал работу?"})

So far we needed to create a separate pipeline for each task. It may be inconvenient if you want to serve multiple prompts simultaneously. For this case we provide a more low-level interface, called MultiPrompt:

In [ ]:
from ruprompts import MultiPrompt

mp = MultiPrompt({
    "detox": "konodyuk/prompt_rugpt3large_detox_russe",
    "qa": "konodyuk/prompt_rugpt3large_qa_sberquad",
})
mp.patch(model, tokenizer)


Now we need to create a regular generation pipeline:

In [ ]:
ppln = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0, return_full_text=False, do_sample=True, eos_token_id=0, max_new_tokens=100)

To switch the task, we just need to change the key:

In [ ]:
ppln(mp(key="detox", toxic_comment="Какого черта ты сюда припёрся?"))

In [ ]:
ppln(mp(key="qa", context=context, question="В каком году была основана Nnaisense?"))